**Run All 하시고 녹음 후 잠시 기다려주세요 ~~**

##모델 불러오기

In [1]:
!git clone https://github.com/AJH0625/Speech-Accent.git
!unzip /content/Speech-Accent/Data_augmented.zip

Cloning into 'Speech-Accent'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [3]:
# Importing required libraries 
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
from tqdm import tqdm
import pickle
import IPython.display as ipd  # To play sound in the notebook

In [4]:
# loading json and model architecture 
json_file = open('Speech-Accent/model128_json_aug.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("Speech-Accent/Model128.h5")
print("Loaded model from disk")

Loaded model from disk


In [5]:
!pip install -q omegaconf torchaudio pydub

import os
from os.path import exists

if not exists('silero-models'):
  !git clone -q --depth 1 https://github.com/snakers4/silero-models

%cd silero-models

# silero imports
import torch
import random
from glob import glob
from omegaconf import OmegaConf
from src.silero.utils import (init_jit_model, 
                       split_into_batches,
                       read_audio,
                       read_batch,
                       prepare_model_input)
from colab_utils import (record_audio,
                         audio_bytes_to_np,
                         upload_audio)

device = torch.device('cpu')   # you can use any pytorch device
models = OmegaConf.load('models.yml')

# imports for uploading/recording
import numpy as np
import ipywidgets as widgets
from scipy.io import wavfile
from IPython.display import Audio, display, clear_output
from torchaudio.functional import vad


# wav to text method
def wav_to_text(f='test.wav'):
  batch = read_batch([f])
  input = prepare_model_input(batch, device=device)
  output = model(input)
  return decoder(output[0].cpu())

     |████████████████████████████████| 79 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 66.4 MB/s 
     |████████████████████████████████| 117 kB 55.4 MB/s 
/content/silero-models


In [6]:
language = "English"
print(language)
if language == 'German':
  model, decoder = init_jit_model(models.stt_models.de.latest.jit, device=device)
elif language == "Spanish":
  model, decoder = init_jit_model(models.stt_models.es.latest.jit, device=device)
else:
  model, decoder = init_jit_model(models.stt_models.en.latest.jit, device=device)

use_VAD = "No"

English


  0%|          | 0.00/112M [00:00<?, ?B/s]

In [7]:
df=pd.read_csv("/content/Data_augmented.csv")
freq=df['labels'].value_counts()
freq_val=freq[freq>=100].index
df=df[df['labels'].isin(freq_val)]
df=df.drop(df[df['labels']=='english'].sample(frac=0.7).index)
X_train, X_test, y_train, y_test = train_test_split(df.drop(['path','labels','source'],axis=1)
                                                    , df.labels, test_size=0.2, shuffle=True, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(lb.classes_)

##내 영어발음은 어느 나라 악센트일까?##
**셀 실행 후 Record Speech 버튼을 눌러 녹음을 시작하세요! 다음 스크립트를 읽으시면 됩니다.**

# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.

In [9]:
!mkdir ../test
record_or_upload = "Record"
record_seconds = 30
sample_rate = 16000

def _apply_vad(audio, boot_time=0, trigger_level=9, **kwargs):
  print('\nVAD applied\n')
  vad_kwargs = dict(locals().copy(), **kwargs)
  vad_kwargs['sample_rate'] = sample_rate
  del vad_kwargs['kwargs'], vad_kwargs['audio']
  audio = vad(torch.flip(audio, ([0])), **vad_kwargs)
  return vad(torch.flip(audio, ([0])), **vad_kwargs)

def _recognize(audio):
  display(Audio(audio, rate=sample_rate, autoplay=True))
  if use_VAD == "Yes":
    audio = _apply_vad(audio)
  wavfile.write('test.wav', sample_rate, (32767*audio).numpy().astype(np.int16))
  transcription = wav_to_text()
  print('\n\nTRANSCRIPTION:\n')
  print(transcription)

def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds)
  wavfile.write('korean.wav', sample_rate, (32767*audio).numpy().astype(np.int16))
  _recognize(audio)

def _upload_audio(b):
  clear_output()
  audio = upload_audio()
  _recognize(audio)
  return audio

if record_or_upload == "Record":
  button = widgets.Button(description="Record Speech")
  button.on_click(_record_audio)
  display(button)
else:
  audio = _upload_audio("")

Starting recording for 5 seconds...


<IPython.core.display.Javascript object>

Finished recording!




TRANSCRIPTION:




##변환중 ...

In [10]:
!cp /content/silero-models/korean.wav /content/test/

DATA='/content/test/'
dir_list=os.listdir(DATA)

nation=[]
path=[]
for i in dir_list:
  new_i=i[:-4]
  new_i=[j for j in new_i if j.isalpha()]
  new_i=''.join(new_i)
  nation.append(new_i)
  path.append(DATA+i)

DATA_df=pd.DataFrame(nation, columns=['labels'])
DATA_df['source']='DATA'
DATA_df=pd.concat([DATA_df, pd.DataFrame(path, columns=['path'])], axis=1)
DATA_df.labels.value_counts()
DATA_df.to_csv("Data_test.csv",index=False)

In [11]:
ref=pd.read_csv("Data_test.csv")
df=pd.DataFrame(columns=['feature'])
cnt=0

for i in tqdm(ref.path):
  X, sample_rate = librosa.load(i, res_type='kaiser_fast', duration=2.5, sr=44100, offset=0.5)
  mfccs=np.mean(librosa.feature.mfcc(y=X,
                                     sr=np.array(sample_rate), n_mfcc=13), axis=0)
  df.loc[cnt]=[mfccs]
  cnt +=1

df=pd.concat([ref, pd.DataFrame(df['feature'].values.tolist())],axis=1)
df=df.fillna(0)
df.to_csv("TEST_processed.csv",index=False)
df.head()

100%|██████████| 1/1 [00:00<00:00,  1.22it/s]


,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,korean,DATA,/content/test/korean.wav,-38.924789,-38.498482,-35.51762,-35.935726,-37.524109,-34.646877,-32.061253,...,-48.054054,-45.594364,-43.538757,-43.631893,-43.477318,-45.279819,-48.855778,-49.310993,-49.672714,-49.555824


In [16]:
test=df.drop(['path','labels','source'],axis=1)
for i in test:
  test[i]=test[i]*0.01
test=np.array(test)
test=np.expand_dims(test, axis=2)

1/1 [==============================] - 0s 31ms/step


,predictedvalues
0,arabic


##RESULT

label: English, Spanish, Arabic, Mandarin, French, Korean, Portuguese, Russian, Dutch, Turkish, German, Polish, Italian, Japanese, Macedonian

In [ ]:
# predictions 
preds=loaded_model.predict(test, verbose=1)
preds=preds.argmax(axis=1)

preds = preds.astype(int).flatten()
preds = (lb.inverse_transform((preds)))
preds = pd.DataFrame({'predictedvalues': preds})

preds